## Informer Demo

In [ ]:
!python -v

## Download code and dataset

In [ ]:
!git clone https://github.com/zhouhaoyi/Informer2020.git
!git clone https://github.com/vanlong32/ETDataset.git
!ls

fatal: destination path 'Informer2020' already exists and is not an empty directory.
fatal: destination path 'ETDataset' already exists and is not an empty directory.
ETDataset  Informer2020  informer_checkpoints  sample_data


In [3]:
# Module sys trong Python cung cấp các hàm và các biến được sử dụng để thao tác các phần khác nhau của môi trường chạy Python.
# Nó cho phép chúng ta truy cập các tham số và chức năng cụ thể của hệ thống.
import sys

# nếu không tìm thấy thư mục Informer2020 thì sửa lại đường dẫn
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

In [3]:
# cài này dùng để cài tất cả thư viện cần thiết
# !pip install -r ./Informer2020/requirements.txt

## Cài đặt cấu hình

**args là gì? args** chỉ là một biến lưu cấu hình giống JSON. Sau đó sẽ được truyền vào exp = Exp(args)



In [1]:
# nhập từ tư viện informer2000, không biết để làm gì 
from utils.tools import dotdict

# nhập từ tư viện informer2000, không biết để làm gì 
from exp.exp_informer import Exp_Informer

# nhập thư viện PyTorch là một framework được xây dựng dựa trên python cung cấp nền tảng 
# tính toán khoa học phục vụ lĩnh vực Deep learning
import torch

In [2]:
args = dotdict()

args.model = 'informer' # model của thử nghiệm, tùy chọn: [informer, informerstack, informerlight(TBD)]

# Tên bộ dữ liệu, nếu muốn dùng datacustom thì phải ghi là  custom
# Tên dataset sẽ được dùng. Giá trị chấp nhận ETTh1, ETTh2, ,ETTm1, ETTm2, custom
args.data = 'custom' 

args.root_path = './ETDataset/ETT-small/' # root path of data file

# ETT là bộ dữ liệu của nhà máy điện
# ETTm1.csv, ETTm2.csv là Bộ dữ liệu ghi theo phút  (ký hiệu m)
# ETTh1.csv, ETTh2.csv là Bộ dữ liệu ghi theo giờ (ký hiệu bằng h)
args.data_path = 'ETTh1.csv' 


# forecasting task, tùy chọn:[M, S, MS]; 
#   - M:multivariate predict multivariate (đa biến dự đoán đa biến)
#   - S:univariate predict univariate (đơn biến dự đoán đơn biến)
#   - MS:multivariate predict univariate (dự đoán đa biến đơn biến)

args.features = 'M' 

# lựa chọn mục tiêu trong chế độ S or MS task
args.target = 'OT' 

# freq for time features encoding (tần số cho nhiệm vụ dự đoán)
#   tùy chọn :[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], 
#   bạn cũng có thể sử dụng freq chi tiết hơn như 15 phút hoặc 3 giờ

args.freq = 'h' 

# Cài vị trí lưu model checkpoints
args.checkpoints = './informer_checkpoints' 

# input sequence length of Informer encoder (độ dài chuỗi đầu vào của bộ mã hóa Informer)
args.seq_len = 96 

# start token length of Informer decoder (độ dài mã thông báo bắt đầu của bộ giải mã Inform)
args.label_len = 48 

# prediction sequence length (độ dài chuỗi dự đoán)
args.pred_len = 24



# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 7 # encoder input size
args.dec_in = 7 # decoder input size
args.c_out = 7 # output size
args.factor = 5 # probsparse attn factor (hệ số suy giảm xác suất)
args.d_model = 512 # dimension of model (kích thước của mô hình)
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention (chú ý) used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding (tính năng mã hóa thời gian), options:[timeF, fixed, learned]
args.activation = 'gelu' # activation (activation)
args.distil = True # whether to use distilling in encoder (có nên sử dụng phương pháp chưng cất trong bộ mã hóa không)
args.output_attention = False # whether to output attention in ecoder (có tạo sự chú ý trong bộ sinh thái không)
args.mix = True
args.padding = 0
args.freq = 'h'

args.batch_size = 32 
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = True
args.devices = '0,1,2,3'


In [4]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

In [5]:
# Set augments by using data name
data_parser = {
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
}

# Date,Open,High,Low,Close,Adj Close,Volume
# data: nhập tên data
# T: là tên cột lựa chọn làm mục tiêu dự đoán
# M
#data_parser = {
#     'BTC-USD':{'data':'BTC-USD.csv','T':'Close','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]}
#}
    


if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

In [8]:
print(args.data)

ETTh1


In [8]:
args.detail_freq = args.freq
args.freq = args.freq[-1:]

### Custom Data - Tùy chỉnh dữ liệu

Custom data (xxx.csv) has to include at least 2 features: `date`(format: `YYYY-MM-DD hh:mm:ss`) and `target feature`.

Dịch: Dữ liệu tùy chỉnh (xxx.csv) phải bao gồm ít nhất 2 tính năng: ngày (định dạng: YYYY-MM-DD hh: mm: ss) và tính năng mục tiêu.

In [9]:
from data.data_loader import Dataset_Custom
from torch.utils.data import DataLoader
import pandas as pd
import os

In [10]:
# custom data: xxx.csv
# data features: ['date', ...(other features), target feature]

# we take ETTh2 as an example
args.root_path = './ETDataset/ETT-small/'
args.data_path = 'ETTh2.csv'

df = pd.read_csv(os.path.join(args.root_path, args.data_path))

In [11]:
df.head()

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,41.130001,12.481,36.535999,9.355,4.424,1.311,38.661999
1,2016-07-01 01:00:00,37.528000,10.136,33.936001,7.532,4.435,1.215,37.124001
2,2016-07-01 02:00:00,37.946999,11.309,35.330002,9.007,2.100,0.000,36.465000
3,2016-07-01 03:00:00,38.952000,11.895,35.543999,9.436,3.380,1.215,33.608501
4,2016-07-01 04:00:00,38.113998,11.476,35.410000,9.623,2.036,0.000,31.850500


In [12]:
'''
We set 'HULL' as target instead of 'OT'

The following frequencies are supported:
        Y   - yearly
            alias: A
        M   - monthly
        W   - weekly
        D   - daily
        B   - business days
        H   - hourly
        T   - minutely
            alias: min
        S   - secondly
'''

args.target = 'HULL'
args.freq = 'h'

Data = Dataset_Custom
timeenc = 0 if args.embed!='timeF' else 1
flag = 'test'; shuffle_flag = False; drop_last = True; batch_size = 1

data_set = Data(
    root_path=args.root_path,
    data_path=args.data_path,
    flag=flag,
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    timeenc=timeenc,
    target=args.target, # HULL here
    freq=args.freq # 'h': hourly, 't':minutely
)
data_loader = (
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    drop_last=drop_last)

In [13]:
batch_x,batch_y,batch_x_mark,batch_y_mark = data_set[0]

### In ra cấu hình
tất cả code phía trên chỉ là khai báo cấu hình bằng text. 

Bên dưới là thông tin cấu hình: 
```
{
  'model': 'informer',
  'data': 'ETTh2', 
  'root_path': './ETDataset/ETT-small/', 
  'data_path': 'ETTh2.csv', 
  'features': 'M', 
  'target': 'OT', 
  'freq': 'h', 
  'checkpoints': './informer_checkpoints', 
  'seq_len': 96, 
  'label_len': 48, 
  'pred_len': 24, 
  'enc_in': 7, 
  'dec_in': 7, 
  'c_out': 7, 
  'factor': 5, 
  'd_model': 512, 
  'n_heads': 8, 
  'e_layers': 2, 
  'd_layers': 1, 
  'd_ff': 2048, 
  'dropout': 0.05, 
  'attn': 'prob', 
  'embed': 'timeF', 
  'activation': 'gelu', 
  'distil': True, 
  'output_attention': False, 
  'mix': True, 
  'padding': 0, 
  'batch_size': 32, 
  'learning_rate': 0.0001, 
  'loss': 'mse', 
  'lradj': 'type1', 
  'use_amp': False, 
  'num_workers': 0, 
  'itr': 1, 
  'train_epochs': 6, 
  'patience': 3, 
  'des': 'exp', 
  'use_gpu': False, 
  'gpu': 0, 
  'use_multi_gpu': False, 
  'devices': '0,1,2,3'
}
```



In [14]:
print('Tất cả các Args (thông số cấu hình):')
print(args)

Tất cả các Args (thông số cấu hình):
{'model': 'informer', 'data': 'ETTh1', 'root_path': './ETDataset/ETT-small/', 'data_path': 'ETTh2.csv', 'features': 'M', 'target': 'HULL', 'freq': 'h', 'checkpoints': './informer_checkpoints', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'enc_in': 7, 'dec_in': 7, 'c_out': 7, 'factor': 5, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'batch_size': 32, 'learning_rate': 0.0001, 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 1, 'train_epochs': 6, 'patience': 3, 'des': 'exp', 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'detail_freq': 'h'}


## Thí nghiệm: Train and Test

In [ ]:
Exp = Exp_Informer

In [ ]:
# args.itr = 1, đây có thể là vòng lặp đào tạo nhiều lần
for ii in range(args.itr):
    # setting record của thí nghiệm
     
    # Cái này dùng để tạo tên để lưu checkpoint, gồm các thông số cấu hình lưu vào tên 
    # Loại: str 

    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

    # Tạo lớp huấn luyện và truyền đối số
    # bên trong chạy hàm khởi tạo. chạy qua 3 dòng code không có gì đặc biệt
 
    exp = Exp(args)
    
    # train
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    
    # khởi động hàm train
    exp.train(setting)
    
    # test
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)

    torch.cuda.empty_cache()

Use CPU
>>>>>>>start training : informer_ETTh2_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8521
val 2857
test 2857


## Prediction

In [ ]:
import os

# Đường dẫn lưu mô hình
setting = 'informer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0'
# path = os.path.join(args.checkpoints,setting,'checkpoint.pth')

In [ ]:
# If you already have a trained model, you can set the arguments and model path, then initialize a Experiment and use it to predict
# Prediction is a sequence which is adjacent to the last date of the data, and does not exist in the data
# If you want to get more information about prediction, you can refer to code `exp/exp_informer.py function predict()` and `data/data_loader.py class Dataset_Pred`

exp = Exp(args)

exp.predict(setting, True)

In [ ]:
# the prediction will be saved in ./results/{setting}/real_prediction.npy
import numpy as np

prediction = np.load('./results/'+setting+'/real_prediction.npy')

prediction.shape

### Thông tin chi tiết về dự đoán và - prediction function

In [ ]:
# Đây là code chi tiết của function predict

def predict(exp, setting, load=False):
    pred_data, pred_loader = exp._get_data(flag='pred')
        
    if load:
        path = os.path.join(exp.args.checkpoints, setting)
        best_model_path = path+'/'+'checkpoint.pth'
        exp.model.load_state_dict(torch.load(best_model_path))

    exp.model.eval()
        
    preds = []
        
    for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(pred_loader):
        batch_x = batch_x.float().to(exp.device)
        batch_y = batch_y.float()
        batch_x_mark = batch_x_mark.float().to(exp.device)
        batch_y_mark = batch_y_mark.float().to(exp.device)

        # decoder input
        if exp.args.padding==0:
            dec_inp = torch.zeros([batch_y.shape[0], exp.args.pred_len, batch_y.shape[-1]]).float()
        elif exp.args.padding==1:
            dec_inp = torch.ones([batch_y.shape[0], exp.args.pred_len, batch_y.shape[-1]]).float()
        else:
            dec_inp = torch.zeros([batch_y.shape[0], exp.args.pred_len, batch_y.shape[-1]]).float()
        dec_inp = torch.cat([batch_y[:,:exp.args.label_len,:], dec_inp], dim=1).float().to(exp.device)
        # encoder - decoder
        if exp.args.use_amp:
            with torch.cuda.amp.autocast():
                if exp.args.output_attention:
                    outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                else:
                    outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        else:
            if exp.args.output_attention:
                outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
            else:
                outputs = exp.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        f_dim = -1 if exp.args.features=='MS' else 0
        batch_y = batch_y[:,-exp.args.pred_len:,f_dim:].to(exp.device)
        
        pred = outputs.detach().cpu().numpy()#.squeeze()
        
        preds.append(pred)

    preds = np.array(preds)
    preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
    
    # result save
    folder_path = './results/' + setting +'/'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    np.save(folder_path+'real_prediction.npy', preds)
    
    return preds


In [ ]:
# bạn có thể sử dụng prediction function để lấy kết quả dự đoán
prediction = predict(exp, setting, True)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(prediction[0,:,-1])
plt.show()

### Thêm chi tiết về Dự đoán - prediction dataset

You can give a `root_path` and `data_path` of the data you want to forecast, and set `seq_len`, `label_len`, `pred_len` and other arguments as other Dataset. The difference is that you can set a more detailed freq such as `15min` or `3h` to generate the timestamp of prediction series.

`Dataset_Pred` only has one sample (including `encoder_input: [1, seq_len, dim]`, `decoder_token: [1, label_len, dim]`, `encoder_input_timestamp: [1, seq_len, date_dim]`, `decoder_input_timstamp: [1, label_len+pred_len, date_dim]`). It will intercept the last sequence of the given data (seq_len data) to forecast the unseen future sequence (pred_len data).

**Dịch:**

Bạn có thể cung cấp đường dẫn gốc và đường dẫn dữ liệu của dữ liệu bạn muốn dự báo, đồng thời đặt seq_len, label_len, pred_len và các đối số khác làm Tập dữ liệu khác. Sự khác biệt là bạn có thể đặt tần suất chi tiết hơn chẳng hạn như 15 phút hoặc 3 giờ để tạo dấu thời gian của chuỗi dự đoán. Dataset_Pred chỉ có một mẫu (bao gồm encoder_input: [1, seq_len, dim], decoder_token: [1, label_len, dim], encoder_input_timestamp: [1, seq_len, date_dim], decoder_input_timstamp: [1, label_len + pred_len, date_dim]). Nó sẽ chặn chuỗi cuối cùng của dữ liệu đã cho (dữ liệu seq_len) để dự báo chuỗi tương lai không nhìn thấy (dữ liệu pred_len).

In [ ]:
from data.data_loader import Dataset_Pred
from torch.utils.data import DataLoader

In [ ]:
Data = Dataset_Pred
timeenc = 0 if args.embed!='timeF' else 1
flag = 'pred'; shuffle_flag = False; drop_last = False; batch_size = 1

freq = args.detail_freq

data_set = Data(
    root_path=args.root_path,
    data_path=args.data_path,
    flag=flag,
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    target=args.target,
    timeenc=timeenc,
    freq=freq
)
data_loader = DataLoader(
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    drop_last=drop_last)

In [ ]:
len(data_set), len(data_loader)

## Visualization - Biểu đồ dữ liệu

In [ ]:
# When we finished exp.train(setting) and exp.test(setting), we will get a trained model and the results of test experiment
# Dịch: Khi chúng tôi hoàn thành exp.train (cài đặt) và exp.test (cài đặt), chúng tôi sẽ nhận được một mô hình được đào tạo và kết quả của thử nghiệm kiểm tra
# The results of test experiment will be saved in ./results/{setting}/pred.npy (prediction of test dataset) and ./results/{setting}/true.npy (groundtruth of test dataset)

preds = np.load('./results/'+setting+'/pred.npy')
trues = np.load('./results/'+setting+'/true.npy')

# [samples, pred_len, dimensions]
preds.shape, trues.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# draw OT prediction
plt.figure()
plt.plot(trues[0,:,-1], label='GroundTruth')
plt.plot(preds[0,:,-1], label='Prediction')
plt.legend()
plt.show()

In [ ]:
# draw HUFL prediction
plt.figure()
plt.plot(trues[0,:,0], label='GroundTruth')
plt.plot(preds[0,:,0], label='Prediction')
plt.legend()
plt.show()

In [ ]:
from data.data_loader import Dataset_ETT_hour
from torch.utils.data import DataLoader

Data = Dataset_ETT_hour
timeenc = 0 if args.embed!='timeF' else 1
flag = 'test'; shuffle_flag = False; drop_last = True; batch_size = 1

data_set = Data(
    root_path=args.root_path,
    data_path=args.data_path,
    flag=flag,
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    timeenc=timeenc,
    freq=args.freq
)
data_loader = DataLoader(
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    drop_last=drop_last)

In [ ]:
import os

args.output_attention = True

exp = Exp(args)

model = exp.model

setting = 'informer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0'
path = os.path.join(args.checkpoints,setting,'checkpoint.pth')
model.load_state_dict(torch.load(path))

In [ ]:
# attention visualization
idx = 0
for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(data_loader):
    if i!=idx:
        continue
    batch_x = batch_x.float().to(exp.device)
    batch_y = batch_y.float()

    batch_x_mark = batch_x_mark.float().to(exp.device)
    batch_y_mark = batch_y_mark.float().to(exp.device)
    
    dec_inp = torch.zeros_like(batch_y[:,-args.pred_len:,:]).float()
    dec_inp = torch.cat([batch_y[:,:args.label_len,:], dec_inp], dim=1).float().to(exp.device)
    
    outputs,attn = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

In [ ]:
attn[0].shape, attn[1].shape #, attn[2].shape

In [ ]:
layer = 0
distil = 'Distil' if args.distil else 'NoDistil'
for h in range(0,8):
    plt.figure(figsize=[10,8])
    plt.title('Informer, {}, attn:{} layer:{} head:{}'.format(distil, args.attn, layer, h))
    A = attn[layer][0,h].detach().cpu().numpy()
    ax = sns.heatmap(A, vmin=0, vmax=A.max()+0.01)
    plt.show()

In [ ]:
layer = 1
distil = 'Distil' if args.distil else 'NoDistil'
for h in range(0,8):
    plt.figure(figsize=[10,8])
    plt.title('Informer, {}, attn:{} layer:{} head:{}'.format(distil, args.attn, layer, h))
    A = attn[layer][0,h].detach().cpu().numpy()
    ax = sns.heatmap(A, vmin=0, vmax=A.max()+0.01)
    plt.show()